In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.precompile()

In [ ]:
using JuMP
using HiGHS

# Data

T = 1:24                    # time steps (example: 24 hours)

# CREATE Q AND D AS ARRAYS SO THAT DEMAND/HEAT VARIES PER HOUR OR PER DAY IDK

Q_data = fill(1000.0, 24)     # Data center waste heat [kW] at each t
D_th = fill(800.0, 24)         # Greenhouse demand [kW] at each t
alpha = 1.5                     # Water evaporation factor [L/kWh rejected]

# Model

model = Model(HiGHS.Optimizer)

@variable(model, x_reuse[t in T] >= 0)       # heat reused in CEA [kW]
@variable(model, x_reject[t in T] >= 0)     # heat rejected to cooling tower [kW]

# Objective
@objective(model, Min, sum(alpha * x_reject[t] for t in T))

# Constraints
@constraint(model, [t in T], x_reuse[t] + x_reject[t] == Q_data[t])
@constraint(model, [t in T], x_reuse[t] <= D_th[t])

optimize!(model)


# This is so obviously AI change this
println("Optimal total water use [L]: ", objective_value(model))

println("\nHour  x_reuse[kW]  x_reject[kW]")

for t in T
    println(rpad(t, 5), " ",
            rpad(value(x_reuse[t]), 12), " ",
            rpad(value(x_reject[t]), 12))
end


Running HiGHS 1.12.0 (git hash: 755a8e027): Copyright (c) 2025 HiGHS under MIT licence terms
LP has 48 rows; 48 cols; 72 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 1e+00]
  Cost    [2e+00, 2e+00]
  Bound   [0e+00, 0e+00]
  RHS     [8e+02, 1e+03]
Presolving model
0 rows, 0 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve reductions: rows 0(-48); columns 0(-48); nonzeros 0(-72) - Reduced to empty
Performed postsolve
Solving the original LP from the solution after postsolve

Model status        : Optimal
Objective value     :  7.2000000000e+03
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00
Optimal total water use [L]: 7200.0

Hour  x_reuse[kW]  x_reject[kW]
1     800.0        200.0       
2     800.0        200.0       
3     800.0        200.0       
4     800.0        200.0       
5     800.0        200.0       
6     800.0        200.0       
7     800.0        200.0       
8     800.0        200.0       
9     800.0        200.0       
10  